# Case - Preparando um dataset para o cálculo de indicadores RFM de uma empresa de e-commerce

## Introdução

Uma empresa do ramo de e-commerce tem uma base de dados contendo diversas informações dos seus clientes ao redor do mundo. Fazendo uma análise criteriosa, esta deseja levantar os indicadores RFM destes clientes, sendo RFM definido por:

- R (Recency): Tempo que o cliente realizou a última compra (em dias)
- F (Frequency): Quantidade de compras realizadas pelo cliente
- M (Monetary): Valor do ticket médio gasto pelo cliente. Onde ticket médio = média do total gasto por pedido para cada cliente.

Além disso, necessita também de gráficos contendo informações valiosas de valores de venda por país, por produto e por mês.

## Objetivo do projeto

A partir da base de dados disponibilizada pela empresa, gerar um output em csv contendo apenas a identificação dos clientes e as métricas RFM calculadas. E, plotar os seguintes gráficos solicitados:

- Top 10 países com maior valor em vendas;
- Top 10 produtos mais vendidos;
- Valor de venda total por mês;
- Valor de venda total por mês e por país (considerando apenas o top 10).

## Sobre os dados

A tabela contém informações de compras de um e-commerce em 37 países. Contém a identificação do cliente e os dados da compra.

- CustomerID = Código de identificação do cliente;
- Description = Descrição do produto;
- InvoiceNo = Código da fatura;
- StockCode = Código de estoque do produto;
- Quantity = Quantidade do produto;
- InvoiceDate = Data do faturamento (compra);
- UnitPrice = Preço unitário do produto;
- Country = País da compra.